<a href="https://colab.research.google.com/github/gupta24789/pytorch_lighting_tutorials/blob/main/1_train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q pytorch_lightning

     |████████████████████████████████| 585 kB 25.1 MB/s 
     |████████████████████████████████| 596 kB 54.9 MB/s 
     |████████████████████████████████| 419 kB 48.8 MB/s 
     |████████████████████████████████| 140 kB 54.7 MB/s 
     |████████████████████████████████| 1.1 MB 42.5 MB/s 
     |████████████████████████████████| 94 kB 1.7 MB/s 
     |████████████████████████████████| 144 kB 43.3 MB/s 
     |████████████████████████████████| 271 kB 62.9 MB/s 


In [3]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl

In [ ]:
## Define Model
class Encoder(nn.Module):
  def __init__(self):
    super.__init__(Encoder, self)
    self.l1 = nn.Sequential()